In [ ]:
# Install necessary libraries
!pip install transformers datasets

In [ ]:
# Import libraries
import torch
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling

In [ ]:
# Load the pre-trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [ ]:
# Load and prepare your custom email dataset (replace with your actual dataset)
# For demonstration purposes, we're using a publicly available dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

In [ ]:
# Tokenization: Apply GPT-2 tokenizer on the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

In [ ]:
# Apply the tokenizer to the dataset
tokenized_dataset = dataset['train'].map(tokenize_function, batched=True, remove_columns=["text"])

In [ ]:
# Prepare data collator for language modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
# Set training arguments
training_args = TrainingArguments(
    output_dir="./results/gpt2_finetuned",  # Model output directory
    overwrite_output_dir=True,
    evaluation_strategy="steps",
    num_train_epochs=3,  # Number of epochs
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    logging_dir='./logs',  # Logging directory
    logging_steps=200,  # Log every 200 steps
    save_steps=500,  # Save every 500 steps
    save_total_limit=2,
    fp16=True,  # Mixed precision training for faster performance
)

In [ ]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

In [ ]:
# Fine-tune GPT-2 model
trainer.train()

In [ ]:
# Save the fine-tuned model and tokenizer
model.save_pretrained("./models/fine_tuned_gpt2")
tokenizer.save_pretrained("./models/fine_tuned_gpt2")

In [ ]:
# Load the fine-tuned GPT-2 model for text generation
fine_tuned_model = GPT2LMHeadModel.from_pretrained("./fine_tuned_gpt2")
fine_tuned_tokenizer = GPT2Tokenizer.from_pretrained("./fine_tuned_gpt2")

In [ ]:
# Generate text using the fine-tuned GPT-2 model
context = "Dear team, I wanted to follow up on"
inputs = fine_tuned_tokenizer(context, return_tensors="pt")

In [ ]:
# Generate text based on context
output = fine_tuned_model.generate(inputs['input_ids'], max_length=100, temperature=0.7, top_p=0.9)


In [ ]:
# Decode the generated tokens to text
generated_text = fine_tuned_tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Email Template:", generated_text)